In [ ]:
import os
import shutil

def flatten_folder(input_dir, output_dir, delimiter='→'):
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Source directory '{input_dir}' does not exist.")
    os.makedirs(output_dir, exist_ok=True)
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file == '': continue
            src_path = os.path.join(root, file)
            subdir_path = os.path.relpath(root, input_dir)
            new_file_name = file if subdir_path == '.' else f"{delimiter.join(filter(None, subdir_path.split(os.sep)))}{delimiter}{file}"
            dest_path = os.path.join(output_dir, new_file_name)
            shutil.copy2(src_path, dest_path)

# Usage
input_dir = ''
output_dir = ''
flatten_folder(input_dir, output_dir, delimiter='→')


In [ ]:
import os
import shutil
import time
from PIL import Image

class ImageCompressor:
    
    def __init__(self, input_dir, output_dir=None, suffix='cmp', quality=90, noSuffix=False, errorCopy=True):
        self.input_dir = input_dir
        self.output_dir = output_dir if output_dir else f"{input_dir}-{suffix}{quality}"
        self.suffix = suffix
        self.quality = quality
        self.noSuffix = noSuffix
        self.errorCopy = errorCopy
        self.stats = {'total_images': 0, 'processed_images': 0, 'original_size': 0, 'compressed_size': 0}
        self.start_time = None

    def compress_images(self):
        # Validate input directory
        if not os.path.exists(self.input_dir):
            raise FileNotFoundError(f"Source directory '{self.input_dir}' does not exist.")

        # Calculate total images and size
        self.calculate_totals()

        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)
        
        self.start_time = time.time()  # Capture the start time

        # Process images
        for root, _, files in os.walk(self.input_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.process_image(file, root)
        
        # Print final stats
        self.print_stats()

    def calculate_totals(self):
        for root, _, files in os.walk(self.input_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.stats['total_images'] += 1
                    self.stats['original_size'] += os.path.getsize(os.path.join(root, file))

    def process_image(self, file, root):
        # Calculate relative path and create output subfolder if needed
        relative_path = os.path.relpath(root, self.input_dir)
        current_output_subdir = os.path.join(self.output_dir, relative_path)
        os.makedirs(current_output_subdir, exist_ok=True)

        try:
            in_path = os.path.join(root, file)
            out_name = f"{os.path.splitext(file)[0]}-{self.suffix}{self.quality}{os.path.splitext(file)[1]}" if not self.noSuffix else file
            out_path = os.path.join(current_output_subdir, out_name)
            with Image.open(in_path) as img:
                img.convert('RGB').save(out_path, optimize=True, quality=self.quality)
                self.stats['processed_images'] += 1
            self.stats['compressed_size'] += os.path.getsize(out_path)
        except Exception as e:
            print(f"\nError processing {file}: {e}")
            if self.errorCopy: shutil.copy2(in_path, current_output_subdir)
        self.print_progress(file)
        
    def print_progress(self, file):
        elapsed_time = time.time() - self.start_time if self.start_time else 0
        remaining_files = self.stats['total_images'] - self.stats['processed_images']
        time_remaining = (elapsed_time / self.stats['processed_images']) * remaining_files if self.stats['processed_images'] > 0 else 0
        print(f"[Elapsed: {elapsed_time:.1f}s | Remaining: {time_remaining:.1f}s] Processed {self.stats['processed_images']}/{self.stats['total_images']} images: {file}\t\t\t\t\t\t\t\t\t\t", end='\r')

    def print_stats(self):
        space_saved_mb = (self.stats['original_size'] - self.stats['compressed_size']) / (1024**2)
        print(f"\nCompression completed in {time.time() - self.start_time:.1f} seconds.")
        print(f"Original size: {self.stats['original_size'] / (1024**2):.2f} MB, Compressed size: {self.stats['compressed_size'] / (1024**2):.2f} MB.")
        print(f"Total space saved: {space_saved_mb:.2f} MB.")

# Usage
input_dir = ''
output_dir = None
compressor = ImageCompressor(input_dir, output_dir, quality=90, noSuffix=False, errorCopy=True)
compressor.compress_images()
